In [11]:
import numpy as np
import pandas as pd
import polapy as po

In [2]:
df = pd.read_stata("data/United States/cumulative_2006-2021.dta")
df["state"] = df["state"].str.upper()
df.head()

,year,case_id,weight,weight_cumulative,state,st,cong,cong_up,state_post,st_post,...,voted_rep_chosen,voted_sen_chosen,voted_gov_chosen,rep_current,rep_icpsr,sen1_current,sen1_icpsr,sen2_current,sen2_icpsr,gov_current
0,2006,439219,1.851676,1.487096,NORTH CAROLINA,NC,109,110,North Carolina,NC,...,Richard C. Carsner (D),,,Patrick T. McHenry (R),20522,Elizabeth Dole (R),40303,Richard Burr (R),29548,Michael Easley (D)
1,2006,439224,0.968308,0.777656,OHIO,OH,109,110,Ohio,OH,...,Stephanie Studebaker (D),Sherrod C. Brown (D),Ted Strickland (D),Michael R. Turner (R),20342,Mike DeWine (R),15020,George V. Voinovich (R),49903,Bob Taft (R)
2,2006,439228,1.593441,1.279706,NEW JERSEY,NJ,109,110,New Jersey,NJ,...,Robert E. Andrews (D),Robert Menendez (D),,Robert E. Andrews (D),29132,Robert Menendez (D),29373,Frank R. Lautenberg (D),14914,Jon Corzine (D)
3,2006,439237,1.398529,1.123170,ILLINOIS,IL,109,110,Illinois,IL,...,Janice D. Schakowsky (D),,Rod Blagojevich (D),Janice D. Schakowsky (D),29911,Richard Durbin (D),15021,Barack Obama (D),40502,Rod Blagojevich (D)
4,2006,439238,0.902890,0.725118,NEW YORK,NY,109,110,New York,NY,...,Maurice D. Hinchey (D),Hillary Rodham Clinton (D),Eliot Spitzer (D),Maurice D. Hinchey (D),29380,Charles E. Schumer (D),14858,Hillary Rodham Clinton (D),40105,George Pataki (R)


In [6]:
print(list(df))

['year', 'case_id', 'weight', 'weight_cumulative', 'state', 'st', 'cong', 'cong_up', 'state_post', 'st_post', 'dist', 'dist_up', 'cd', 'cd_up', 'dist_post', 'dist_up_post', 'cd_post', 'cd_up_post', 'zipcode', 'county_fips', 'tookpost', 'weight_post', 'rvweight', 'rvweight_post', 'starttime', 'pid3', 'pid3_leaner', 'pid7', 'ideo5', 'gender', 'birthyr', 'age', 'race', 'hispanic', 'citizen', 'educ', 'marstat', 'faminc', 'union', 'union_hh', 'employ', 'no_healthins', 'has_child', 'ownhome', 'no_milstat', 'religion', 'economy_retro', 'newsint', 'approval_pres', 'approval_rep', 'approval_sen1', 'approval_sen2', 'approval_gov', 'intent_pres_08', 'intent_pres_12', 'intent_pres_16', 'intent_pres_20', 'voted_pres_08', 'voted_pres_12', 'voted_pres_16', 'voted_pres_20', 'intent_pres_party', 'voted_pres_party', 'intent_turnout_self', 'voted_turnout_self', 'vv_regstatus', 'vv_party_gen', 'vv_party_prm', 'vv_turnout_gvm', 'vv_turnout_pvm', 'intent_rep', 'intent_rep_party', 'voted_rep', 'voted_rep_par

In [53]:
year = "20"
output = []
for year in ["08", 12, 16, 20]:
    candidate = f"intent_pres_{year}"

    data = df[["year", "state", candidate, "weight"]].dropna().groupby(["state", candidate]).agg({"weight": "sum"}).reset_index()
    data = data.rename(columns={"weight": "votes"})
    data = data[data[candidate].isin([
    "Mitt Romney (Republican)", "John McCain", "Barack Obama", "Barack Obama (Democratic)", "Donald Trump (Republican)", "Joe Biden (Democrat)", "Hillary Clinton (Democrat)"])].reset_index()
    data[candidate] = data[candidate].astype(str)
    data.head()

    ep, antagonisms = po.within_ep(
        data,
        unit="state",
        candidate=candidate,
        votes="votes",
        score=None
    )
    output.append({
        "year": int(f"20{year}"),
        "ep": ep
    })

(0.12130995530706677,
               intent_pres_12  antagonism
 0  Barack Obama (Democratic)    0.060089
 1   Mitt Romney (Republican)    0.061221)

In [10]:
df[["year", "state", "intent_pres_20", "weight"]].dropna()

,year,state,intent_pres_20,weight
470756,2020,FLORIDA,Joe Biden (Democrat),1.344424
470757,2020,IOWA,Joe Biden (Democrat),0.405520
470758,2020,MASSACHUSETTS,Joe Biden (Democrat),0.957734
470759,2020,ILLINOIS,Other,0.194665
470760,2020,OHIO,Donald Trump (Republican),1.056914
...,...,...,...,...
531747,2020,CALIFORNIA,Donald Trump (Republican),0.363201
531748,2020,NORTH CAROLINA,Donald Trump (Republican),0.951641
531749,2020,ALABAMA,Donald Trump (Republican),0.496617
531750,2020,PENNSYLVANIA,I Won't Vote in this Election,1.790207


In [49]:
df_filtered = df[["year", "state", "pid7", "weight"]]
# df_filtered = df_filtered[df_filtered["tookpost"] == "Took Post-Election Survey"]
df_filtered["ideology"] = df_filtered["pid7"].replace({
    "Strong Democrat": -3,
    "Not Very Strong Democrat": -2,
    "Lean Democrat": -1,
    "Independent": 0,
    "Lean Republican": 1,
    "Not Very Strong Republican": 2,
    "Strong Republican": 3,
    "Not Sure": np.nan
}).astype(object)

df_filtered = df_filtered[df_filtered["ideology"].isin([-3, -2, -1, 1, 2, 3])]
df_filtered["ideology"] = df_filtered["ideology"].astype(int)
df_filtered["pol_orientation"] = df_filtered["ideology"].apply(lambda x: "LEFT" if x < 0 else "RIGHT")
df_filtered.head()

C:\Users\cnava\AppData\Local\Temp\ipykernel_24704\3088419169.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["ideology"] = df_filtered["pid7"].replace({


,year,state,pid7,weight,ideology,pol_orientation
0,2006,NORTH CAROLINA,Strong Democrat,1.851676,-3,LEFT
1,2006,OHIO,Lean Democrat,0.968308,-1,LEFT
2,2006,NEW JERSEY,Strong Democrat,1.593441,-3,LEFT
3,2006,ILLINOIS,Strong Democrat,1.398529,-3,LEFT
4,2006,NEW YORK,Strong Democrat,0.902890,-3,LEFT


In [54]:
df_ep = pd.DataFrame(output)
df_ep

,year,ep
0,2008,0.127249
1,2012,0.121310
2,2016,0.142289
3,2020,0.157832


In [59]:
data1 = df_filtered.groupby(["year", "pol_orientation"]).agg({"ideology": "mean"}).unstack().reset_index()
data1.columns = ["year", "left", "right"]
data1["pp"] = data1["right"] - data1["left"]
pd.merge(df_ep, data1, on="year")

,year,ep,left,right,pp
0,2008,0.127249,-2.343884,2.315616,4.659500
1,2012,0.121310,-2.328800,2.161104,4.489904
2,2016,0.142289,-2.320531,2.140308,4.460838
3,2020,0.157832,-2.320252,2.246302,4.566554
